# LSTM on Recipe Data

**The notebook has been adapted from the notebook provided in David Foster's Generative Deep Learning, 2nd Edition.**

- Book: [Amazon](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1098134184/ref=sr_1_1?keywords=generative+deep+learning%2C+2nd+edition&qid=1684708209&sprefix=generative+de%2Caps%2C93&sr=8-1)
- Original notebook (tensorflow and keras): [Github](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/05_autoregressive/01_lstm/lstm.ipynb)
- Dataset: [Kaggle](https://www.kaggle.com/datasets/hugodarwood/epirecipes)

In [16]:
import re
import string
import json

import numpy as np
import jax
import jax.numpy as jnp
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import utils

import flax.linen as nn

## 0. Train parameters

In [4]:
DATA_DIR = '../../data/epirecipes/full_format_recipes.json'

EMBEDDING_DIM = 100
HIDDEN_DIM = 128
VALIDATION_SPLIT = 0.2
SEED = 1024
BATCH_SIZE = 32
EPOCHS = 30

MAX_PAD_LEN = 200
MAX_VAL_TOKENS = 100 # Max number of tokens when generating texts

## 1. Load dataset

In [7]:
def pad_punctuation(sentence):
    sentence = re.sub(f'([{string.punctuation}])', r' \1 ', sentence)
    sentence = re.sub(' +', ' ', sentence)
    return sentence

In [10]:
# load dataset
with open(DATA_DIR, 'r+') as f:
    recipe_data = json.load(f)

In [11]:
# preprocess dataset
filtered_data = [
    'Recipe for ' + x['title'] + ' | ' + ' '.join(x['directions'])
    for x in recipe_data
    if 'title' in x and x['title']
    and 'directions' in x and x['directions']
]

text_ds = [pad_punctuation(sentence) for sentence in filtered_data]
print(f'Total recipe loaded: {len(text_ds)}')

Total recipe loaded: 20098


In [14]:
print('Sample data:')
sample_data = np.random.choice(text_ds)
print(sample_data)

Sample data:
Recipe for Ditalini with Pesto , Beans , and Broccoli Rabe | Cook pasta in large pot of boiling salted water 3 minutes . Add broccoli rabe and boil until broccoli rabe is just crisp - tender and pasta is just tender but still firm to bite , stirring occasionally , about 5 minutes longer . Drain pasta and broccoli rabe , reserving 1 / 2 cup cooking liquid . Return pasta and broccoli rabe to pot . Meanwhile , bring vegetable broth and crushed red pepper to simmer in medium saucepan . Add beans and simmer until beans are heated through , stirring frequently , about 5 minutes . Add bean mixture , pesto , and vinegar to pasta and broccoli rabe . Stir well , adding pasta cooking liquid by tablespoonfuls if necessary . Season with salt and pepper . Serve in shallow bowls . 


## 2. Build vocabularies